# Run Hugging Face `Llama 3.1 8B` autoregressive sampling on Inf2 & Trn1 with 32k sequence length

In this example we compile and deploy the Hugging Face [meta-llama/Meta-Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct) model for tensor parallel inference on Neuron using the `transformers-neuronx` package. We use a sequence length of 32k.

The example has the following main sections:
1. Set up the Jupyter Notebook
2. Install dependencies
3. Download the model
4. Perform autoregressive sampling using tensor parallelism

This Jupyter Notebook can be run on an Inf2 instance (`inf2.48xlarge`) or Trn1 instance (`trn1.32xlarge`).

## Set up the Jupyter Notebook

The following steps set up Jupyter Notebook and launch this tutorial:
1. Clone the [AWS Neuron Samples](https://github.com/aws-neuron/aws-neuron-samples) repo to your instance using
```
git clone https://github.com/aws-neuron/aws-neuron-samples.git
```
2. Navigate to the `transformers-neuronx` inference samples folder
```
cd aws-neuron-samples/torch-neuronx/transformers-neuronx/inference
```
3. Follow the instructions in [Jupyter Notebook QuickStart](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/setup/notebook/setup-jupyter-notebook-steps-troubleshooting.html) to run Jupyter Notebook on your instance.
4. Locate this tutorial in your Jupyter Notebook session (`llama-3.1-8b-32k-sampling.ipynb`) and launch it. Follow the rest of the instructions in this tutorial. 

## Install Dependencies
This tutorial requires the following pip packages:

 - `torch-neuronx`
 - `neuronx-cc`
 - `sentencepiece`
 - `transformers`
 - `transformers-neuronx`


Most of these packages will be installed when configuring your environment using the [torch-neuronx inference setup guide](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/general/setup/torch-neuronx.html#setup-torch-neuronx). The additional dependencies must be installed here:

In [ ]:
!pip install transformers-neuronx sentencepiece 
!pip install transformers>=4.43.3 # need recent transformers version for RoPE scaling in Llama 3.1

## Access the model

Use of the Llama 3.1 model is governed by the Meta license and must be downloaded prior to running this sample. Follow the steps described in [meta-llama/Meta-Llama-3.1-8B-Instruct](https://huggingface.co/meta-llama/Meta-Llama-3.1-8B-Instruct) to get access to the Llama 3.1 model from Meta.

Note: For the purpose of this sample, we assume you have access to the model from Hugging Face and it is saved in the directory `Meta-Llama-3.1-8B-Instruct`.

## Perform autoregressive sampling using tensor parallelism

Now we have all of the necessary files for running `meta-llama/Meta-Llama-3.1-8B-Instruct` autoregressive sampling.

The memory required to host any model can be computed with:
```
total memory = bytes per parameter * number of parameters
```
When using `bfloat16` weights for a 8 billion parameter model, this works out to `2 * 8B` or ~16GB of weights. Each NeuronCore has 16GB of memory which means that a 16GB model would just fit on a single NeuronCore. In reality, the total space required is often greater than just the number of parameters due to caching attention layer projections (KV caching). This caching mechanism grows memory allocations linearly with sequence length and batch size.

To get very large language models to fit on Inf2 & Trn1, tensor parallelism is used to split weights, data, and compute across multiple NeuronCores. The number of NeuronCores that the weights are split across can be controlled by setting the `tp_degree` parameter. This parallelism degree must be chosen to ensure that the memory usage per NeuronCore will be less than the physical 16GB limit. When configuring tensor parallelism, the memory per NeuronCore can be computed with:

```
memory per core = (bytes per parameter * number of parameters) / tp_degree
```

This can be used to compute the minimum instance sizing by ensuring that the value selected for `tp_degree` results in less than 16GB allocated per NeuronCore.

Note that increasing the `tp_degree` beyond the minimum requirement almost always results in a faster model. Increasing the tensor parallelism degree improves memory bandwidth which improves model performance. To optimize performance it's recommended to use the highest tensor parallelism degree that is supported by the instance. In this sample we use tensor parallelism degree 24 to optimize performance on `inf2.48xlarge`, but this should be changed to 32 if you are using a `trn1.32xlarge`. 

We will use the Neuron `LlamaForSampling` class to implement tensor parallelism for the Llama based model. We supply the n_positions as 32768 and context length estimates to precompile various possible prompt lengths. Tensor parallelism is enabled through the argument `tp_degree=24`. The model computational graph is compiled by `neuronx-cc` for optimized inference on Neuron.

In [ ]:
## Load the model

from transformers_neuronx import LlamaForSampling

model_path = "Meta-Llama-3.1-8B-Instruct"

# load Meta-Llama-3.1-8B-Instruct to the NeuronCores with 24-way tensor parallelism and run compilation
# we pass n_positions as 32768 so that the model supports 32k sequence length, and pass the context_length_estimate
# list generated below.

# we supply a list of context_length_estimate that allows us to get low context encoding latency even for 
# larger prompts than 16384 (largest context_length_estimate by default is 1/2 of n_positions input)
context_length_estimate = [64, 128, 256, 512, 1024, 2048, 4096, 8192, 16384, 20480, 24576, 28672, 32768]

neuron_model = LlamaForSampling.from_pretrained(model_path, n_positions=32768, \
                                                context_length_estimate=context_length_estimate, \
                                                batch_size=1, tp_degree=24, amp='bf16')

Notice that context encoding buckets are used via `context_length_estimate` to improve the context encoding latency. For more details about how to effectively use bucketing, please refer the [developer guide for bucketing](https://awsdocs-neuron.readthedocs-hosted.com/en/latest/libraries/transformers-neuronx/transformers-neuronx-developer-guide.html?highlight=bucketing#bucketing).

In [ ]:
# Load model on neuron cores and compile
neuron_model.to_neuron()

In [ ]:
## Perform autoregressive sampling

import time
import torch
from transformers import AutoTokenizer
import requests, re

# construct a tokenizer and encode prompt text
# For the prompt we take a recent publication (html format), strip html tags to convert to txt
# and ask the model to summarize the paper. The input length is 26k+ tokens.
tokenizer = AutoTokenizer.from_pretrained(model_path)
prompt = re.sub('<[^<]+?>', '', requests.get("https://arxiv.org/html/2402.19427v1").text) # strip html tags
prompt += "\n\n========================THE END======================\n"
prompt += "A 10 point summary of the paper in simple words: "
# put in prompt format https://llama.meta.com/docs/model-cards-and-prompt-formats/llama3_1/#prompt-format
prompt = f"<|begin_of_text|><|start_header_id|>user<|end_header_id|> {prompt} <|eot_id|><|start_header_id|>assistant<|end_header_id|>"

input_ids = tokenizer.encode(prompt, return_tensors="pt") 
num_input_tokens = len(input_ids[0]) # over 26k tokens
print(f"num_input_tokens: {num_input_tokens}")

# run inference with top-k sampling
with torch.inference_mode():
    start = time.time()
    generated_sequences = neuron_model.sample(input_ids, sequence_length=32768, top_k=10)
    elapsed = time.time() - start

# display the new generated tokens
generated_sequences = [tokenizer.decode(seq[num_input_tokens:]) for seq in generated_sequences]
print(f'generated sequence {generated_sequences[0]} in {elapsed} seconds')

## Save and load the compiled model

The ```save``` and ```load``` functions can be used to save and load compiled model artifacts respectively. Loading compiled model artifacts from a provided directory will avoid model recompilation.

In [ ]:
neuron_model.save('./neuron_artifacts') # can be copied and used on a different neuron instance
del neuron_model

neuron_model = LlamaForSampling.from_pretrained(model_path, n_positions=32768, \
                                                context_length_estimate=context_length_estimate, \
                                                batch_size=1, tp_degree=24, amp='bf16')

neuron_model.load('neuron_artifacts') # Load the compiled Neuron artifacts
neuron_model.to_neuron() # will skip compile

with torch.inference_mode():
    start = time.time()
    generated_sequences = neuron_model.sample(input_ids, sequence_length=32768, top_k=10)
    elapsed = time.time() - start

generated_sequences = [tokenizer.decode(seq[num_input_tokens:]) for seq in generated_sequences]
print(f'generated sequence {generated_sequences[0]} in {elapsed} seconds')